## let's now visualize our data in terms of nutrient

In [ ]:
import pandas as pd
import sqlite3
import re
import numpy as np
import plotly.express as px
import nbformat
import matplotlib.pyplot as plt
from matplotlib.widgets import CheckButtons
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
db_path = "/Users/siddharthgianchandani/Desktop/vscode projects/consumption-gaps/final scraper/trying_morrisons/morrisons_products.db"
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM products", conn)
conn.close()

In [3]:
def extract_kj(energy_str):
    if not isinstance(energy_str, str):
        return np.nan
    # trying to find a number followed by 'kJ'
    match = re.search(r'(\d+)\s*kJ', energy_str)
    if match:
        return int(match.group(1))
    # sometimes just a number
    match = re.search(r'(\d+)', energy_str)
    if match:
        return int(match.group(1))
    return np.nan

def extract_grams(val):
    if not isinstance(val, str):
        return np.nan
    # handle "<0.5g" or similar
    match = re.search(r'([\d\.]+)', val)
    if match:
        return float(match.group(1))
    return np.nan

In [14]:
# cleaning the data

df['energy_kj'] = df['energy_per_100g'].apply(extract_kj)
df['fat_g'] = df['fat_per_100g'].apply(extract_grams)
df['sat_fat_g'] = df['saturated_fat_per_100g'].apply(extract_grams)
df['protein_g'] = df['protein_per_100g'].apply(extract_grams)
df['price'] = df['price'].replace({'£': ''}, regex=True).astype(float)

# drop rows with missing values in any relevant columns
df_clean = df.dropna(subset=['energy_kj', 'fat_g', 'sat_fat_g', 'protein_g', 'category', 'title'])

In [15]:
df_clean['protein_per_energy'] = df_clean['protein_g'] / df_clean['energy_kj'].replace(0, np.nan)

df_clean = df_clean.replace([np.inf, -np.inf], np.nan).dropna(subset=['protein_per_energy'])
df_clean = df_clean.drop(columns=['energy_per_100g', 'fat_per_100g', 'saturated_fat_per_100g', 'protein_per_100g', 'link', 'energy_kj', 'protein_g'])

/var/folders/zz/k81wn_s93cdffqmg3682c6dc0000gn/T/ipykernel_30465/2331283605.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
df_clean

,id,category,title,price,fat_g,sat_fat_g,protein_per_energy
5,6,Orange,Tropicana Original Orange Juice,2.25,0.00,0.00,0.004571
6,7,Orange,Tropicana Multi Vitamins Juice,2.25,0.00,0.00,0.002000
7,8,Orange,Innocent Smooth Orange Juice,3.20,0.00,0.00,0.004545
8,9,Orange,Innocent Orange Juice with Bits,3.20,0.00,0.00,0.005128
11,12,Orange,Innocent Smooth Orange Juice,4.40,0.00,0.00,0.004545
...,...,...,...,...,...,...,...
332,333,"Beer, Wines & Spirits",Trip CBD Infused Raspberry Orange Blossom,5.50,0.00,0.00,0.000000
333,334,"Beer, Wines & Spirits",Goodrays CBD Infused Blood Orange & Grapefruit...,2.50,0.00,0.00,0.000000
336,337,"Beer, Wines & Spirits",Bonne Maman Chocolate & Orange Spread,3.00,4.50,2.60,0.002064
337,338,"Beer, Wines & Spirits",Tassimo Cadbury Orange Hot Chocolate Pods x8,5.30,0.90,0.80,0.007619


In [33]:
categories = df_clean['category'].unique()

fig = go.Figure()
for i, cat in enumerate(categories):
    df_cat = df_clean[df_clean['category'] == cat]
    fig.add_trace(go.Scatter(
        x=df_cat['price'],
        y=df_cat['protein_per_energy'],
        mode='markers',
        marker=dict(
            size=11, 
            color=df_cat['fat_g'],
            colorscale='Reds',
            colorbar=dict(title='Fat (g)'),
            line=dict(width=0)
        ),
        name=cat,
        text=df_cat['title'],
        hovertemplate='<b>%{text}</b><br>Price: %{x}<br>Protein/Energy: %{y}<br>Fat: %{marker.color}<extra></extra>',
        visible=(i == 0)
    ))

buttons = []
for i, cat in enumerate(categories):
    visible = [False] * len(categories)
    visible[i] = True
    buttons.append(dict(
        label=cat,
        method='update',
        args=[{'visible': visible},
              {'title': f'Protein to Energy Ratio vs Price (Category: {cat})'}]
    ))

buttons.insert(0, dict(
    label='All',
    method='update',
    args=[{'visible': visible},
          {'title': 'Protein to Energy Ratio vs Price (All)'}]
))

fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=1.15,
        y=1.15
    )],
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title='Protein to Energy Ratio vs Price (All Categories)',
    xaxis_title='Price',
    yaxis_title='Protein/Energy Ratio',
)
fig.show()